In [27]:
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn import neighbors, datasets, svm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

import matplotlib
#matplotlib.use('GTKAgg')
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap
%matplotlib inline
matplotlib.rcParams.update({'font.size': 18})

pip install xlrd==1.2.0.  (latest version of xlrd only imports .xls 

In [9]:
df = pd.read_excel ('Black-Box-Optimization/Data/All Data combined.xlsx')
                  

<ipython-input-9-428d1058db4a>:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  df = pd.read_excel ('Black-Box-Optimization/Data/All Data combined.xlsx')


In [10]:
df

,Unnamed: 0,number of trials,number of parameters,type of function,accuracy [calc. max/ actual max],time per trial [s],type_of_opt,assigned_class
0,0.0,16,2,Trig,0.732417,0.009824,CmaEs,0
1,1.0,13,2,Trig,0.983980,0.008316,CmaEs,0
2,2.0,10,2,Trig,0.638160,0.007854,CmaEs,0
3,3.0,18,2,Trig,0.952784,0.009196,CmaEs,0
4,4.0,11,2,Trig,0.284646,0.007938,CmaEs,0
...,...,...,...,...,...,...,...,...
1345,NaN,8,2,Trig,0.034904,0.007662,CMA-ES,5
1346,NaN,8,2,Trig,0.091423,0.010617,CMA-ES,5
1347,NaN,14,2,Trig,0.015566,0.019561,CMA-ES,5
1348,NaN,9,2,Trig,0.076210,0.011982,CMA-ES,5


In [6]:
print(len(df))
print(df.head())

1350
   Unnamed: 0  number of trials  number of parameters type of function  \
0         0.0                16                     2             Trig   
1         1.0                13                     2             Trig   
2         2.0                10                     2             Trig   
3         3.0                18                     2             Trig   
4         4.0                11                     2             Trig   

   accuracy [calc. max/ actual max]  time per trial [s] type_of_opt  
0                          0.732417            0.009824       CmaEs  
1                          0.983980            0.008316       CmaEs  
2                          0.638160            0.007854       CmaEs  
3                          0.952784            0.009196       CmaEs  
4                          0.284646            0.007938       CmaEs  


In [7]:
# replace zeroes
zero_not_accepted = ['number of parameters', 'type of function', 'accuracy [calc. max/ actual max]', 'time per trial [s]', 'type_of_opt']

for column in zero_not_accepted:
    df[column] = df[column].replace(0, np.NaN)
    

In [31]:
# split dataset
# X = df.iloc[:,0:6]
# y = df.iloc[:,5:6]
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)

In [11]:
X = df[['accuracy [calc. max/ actual max]','time per trial [s]']]
y = df['assigned_class']

print(len(y))

1350


In [12]:
# split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=14) #14 is seed

print(len(y_train), len(y_test))

1080 270


In [13]:
#Feature scaling
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [14]:
K_neighbors = 1

In [15]:
#Define the model: Init K-NN
# We create an instance of Neighbours Classifier and fit it to the training data.
# if you set weights = 'distance' here you will have a weighted KNN classifier
clf = neighbors.KNeighborsClassifier(K_neighbors, weights='distance')

# Here we fit it to the data - it figures out the classes and 
# which training X points belong to each class in Y 
clf.fit(X, y)

KNeighborsClassifier(n_neighbors=1, weights='distance')

In [16]:
print(clf.classes_, clf.effective_metric_, clf.n_samples_fit_)

[0 1 2 3 4 5] euclidean 1350


In [17]:
# Predict the test set results
y_test_predicted = clf.predict(X_test)
y_test_probabilities = clf.predict_proba(X_test)
mean_accuracy = clf.score(X_test, y_test)

print("predicted classes", y_test_predicted)
print("KNN probability of belonging to class",y_test_probabilities)
print("Mean accuracy of prediction", mean_accuracy)


predicted classes [4 2 2 2 1 2 5 2 4 2 2 2 2 3 2 2 1 2 2 1 5 5 2 2 2 2 2 4 2 1 2 1 1 4 2 2 5
 3 2 2 2 2 2 2 5 2 2 2 2 2 1 2 5 2 5 2 5 4 2 2 2 1 2 2 2 2 2 2 2 4 2 5 3 2
 2 3 2 2 2 2 2 1 2 2 5 2 5 2 2 1 1 2 2 5 2 2 3 3 2 2 5 2 2 2 1 2 2 5 1 2 1
 5 1 2 5 3 2 2 2 2 2 1 4 2 3 2 5 2 1 1 3 2 2 4 2 2 2 4 2 4 2 1 2 2 2 2 4 2
 2 2 1 1 2 3 2 2 2 4 2 5 2 5 2 2 5 3 1 2 4 4 2 3 2 2 2 2 3 3 2 2 2 2 5 2 3
 1 1 2 1 1 1 4 1 2 2 2 2 2 4 2 5 2 2 2 2 2 3 0 2 5 3 2 4 4 3 2 5 2 2 5 2 1
 2 2 5 2 2 2 2 2 5 4 5 2 2 2 1 1 2 3 2 5 2 5 5 5 2 2 2 2 2 1 2 2 2 1 4 3 4
 2 4 0 2 2 1 2 2 1 4 4]
KNN probability of belonging to class [[0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 ...
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]]
Mean accuracy of prediction 0.2851851851851852


In [18]:
X_train[:,0]

array([-0.26415484,  0.2639297 , -0.13229526, ..., -0.012802  ,
       -0.39174869, -0.17265342])

In [19]:
# Evaluate Model
cm = confusion_matrix(y_test, y_test_predicted)
print(cm)

[[ 0  8 48  5  0  0]
 [ 0  7 46  2  1  0]
 [ 0  9 45  2  0  0]
 [ 2 11 10 11  9 32]
 [ 0  0  0  0 14  0]
 [ 0  0  8  0  0  0]]


In [ ]:
# classifier = KNeighborsClassifier(n_neighbors=1, p=2, metric='euclidean')

In [61]:
# Create color maps


In [25]:
model = svm.SVC(kernel='rbf',C=30,gamma='auto')
model.fit(X_train,y_train)
model.score(X_test, y_test)

0.5851851851851851

Hyper parameter Tuning 

In [36]:
kernels = ['rbf','linear']
C = [1,10,20,30,40,50,60,70,80,90,100]
avg_scores = {}
for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(svm.SVC(kernel=kval,C=cval,gamma='auto'), X, df['type_of_opt'], cv=5)
        avg_scores[kval + '_' + str(cval)] = np.average(cv_scores)
        
avg_scores

{'rbf_1': 0.37851851851851853,
 'rbf_10': 0.42518518518518517,
 'rbf_20': 0.454074074074074,
 'rbf_30': 0.46296296296296297,
 'rbf_40': 0.4674074074074074,
 'rbf_50': 0.4696296296296296,
 'rbf_60': 0.47481481481481486,
 'rbf_70': 0.4762962962962963,
 'rbf_80': 0.47925925925925916,
 'rbf_90': 0.48592592592592593,
 'rbf_100': 0.48740740740740734,
 'linear_1': 0.3511111111111111,
 'linear_10': 0.4140740740740741,
 'linear_20': 0.4348148148148148,
 'linear_30': 0.4525925925925926,
 'linear_40': 0.46296296296296297,
 'linear_50': 0.47185185185185186,
 'linear_60': 0.48,
 'linear_70': 0.4851851851851852,
 'linear_80': 0.4948148148148149,
 'linear_90': 0.4992592592592593,
 'linear_100': 0.5059259259259259}

Hyper parameter Tunning with GridSearchCV

In [38]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(svm.SVC(gamma='auto'), {
    'C': [1,10,20,30,40,50,60,70,80,90,100],
    'kernel':['rbf','linear']
}, cv=5, return_train_score=False)

clf.fit(X,df['type_of_opt'])
clf.cv_results_

{'mean_fit_time': array([0.0607007 , 0.03937974, 0.05963721, 0.04295115, 0.06663823,
        0.04720898, 0.05375924, 0.04701262, 0.05817485, 0.05143118,
        0.05913448, 0.05167127, 0.05806718, 0.05237694, 0.05257545,
        0.0573998 , 0.05919542, 0.05795979, 0.06032386, 0.05403357,
        0.05478148, 0.05658808]),
 'std_fit_time': array([0.0063703 , 0.00347135, 0.00153366, 0.00278639, 0.01649685,
        0.00296944, 0.00146151, 0.00241751, 0.00136723, 0.00334215,
        0.00250933, 0.00496032, 0.00396953, 0.00444209, 0.00134918,
        0.00629174, 0.00336004, 0.00415806, 0.00325869, 0.00568504,
        0.00137623, 0.00576497]),
 'mean_score_time': array([0.02472839, 0.00775919, 0.02585788, 0.00859485, 0.02912674,
        0.00839362, 0.02335353, 0.00873981, 0.02577596, 0.00865159,
        0.02611914, 0.00818257, 0.02459989, 0.0080369 , 0.02271605,
        0.00827804, 0.02517457, 0.00821867, 0.02538743, 0.00756354,
        0.02351055, 0.00785751]),
 'std_score_time': array([0.00

In [39]:
df_1 = pd.DataFrame(clf.cv_results_)
df_1


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.060701,0.006370,0.024728,0.000556,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.240741,0.318519,0.433333,0.414815,0.485185,0.378519,0.087502,21
1,0.039380,0.003471,0.007759,0.000454,1,linear,"{'C': 1, 'kernel': 'linear'}",0.233333,0.274074,0.418519,0.362963,0.466667,0.351111,0.086998,22
2,0.059637,0.001534,0.025858,0.001463,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.303704,0.414815,0.481481,0.462963,0.462963,0.425185,0.064636,19
3,0.042951,0.002786,0.008595,0.000558,10,linear,"{'C': 10, 'kernel': 'linear'}",0.300000,0.381481,0.485185,0.433333,0.470370,0.414074,0.067338,20
4,0.066638,0.016497,0.029127,0.005808,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.337037,0.414815,0.503704,0.496296,0.518519,0.454074,0.068757,16
5,0.047209,0.002969,0.008394,0.000488,20,linear,"{'C': 20, 'kernel': 'linear'}",0.377778,0.403704,0.500000,0.425926,0.466667,0.434815,0.043735,18
6,0.053759,0.001462,0.023354,0.000892,30,rbf,"{'C': 30, 'kernel': 'rbf'}",0.359259,0.448148,0.500000,0.492593,0.514815,0.462963,0.056413,14
7,0.047013,0.002418,0.008740,0.001232,30,linear,"{'C': 30, 'kernel': 'linear'}",0.396296,0.440741,0.514815,0.433333,0.477778,0.452593,0.040464,17
8,0.058175,0.001367,0.025776,0.001096,40,rbf,"{'C': 40, 'kernel': 'rbf'}",0.385185,0.451852,0.511111,0.485185,0.503704,0.467407,0.045926,13
9,0.051431,0.003342,0.008652,0.000443,40,linear,"{'C': 40, 'kernel': 'linear'}",0.414815,0.448148,0.529630,0.440741,0.481481,0.462963,0.039545,14


In [41]:
df_1[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.378519
1,1,linear,0.351111
2,10,rbf,0.425185
3,10,linear,0.414074
4,20,rbf,0.454074
5,20,linear,0.434815
6,30,rbf,0.462963
7,30,linear,0.452593
8,40,rbf,0.467407
9,40,linear,0.462963


In [42]:
clf.best_score_

0.5059259259259259

In [43]:
clf.best_params_

{'C': 100, 'kernel': 'linear'}

Randmized search for hyper parameters

In [57]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(svm.SVC(gamma='auto'), {
        'C': np.linspace(0,500,50),
        'kernel':['rbf','linear']
    },
    cv=5, #cross validation
    return_train_score=False,
    n_iter=21
)
rs.fit(X, df['type_of_opt'])
pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]

/Users/saleksegid/miniconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/saleksegid/miniconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/saleksegid/miniconda3/lib/python3.8/site-packages/sklearn/svm/_base.py", line 226, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "/Users/saleksegid/miniconda3/lib/python3.8/site-packages/sklearn/svm/_base.py", line 277, in _dense_fit
    self._probB, self.fit_status_ = libsvm.fit(
  File "sklearn/svm/_libsvm.pyx", line 192, in sklearn.svm._libsvm.fit
ValueError: C <= 0

  warnings.warn("Estimator fit failed. The score on this train-test"
/Users/saleksegid/miniconda3/lib/python3.8/site-packages/

,param_C,param_kernel,mean_test_score
0,336.734694,rbf,0.522222
1,183.673469,rbf,0.497037
2,438.77551,linear,0.528148
3,0.0,linear,NaN
4,316.326531,rbf,0.518519
5,102.040816,linear,0.506667
6,255.102041,rbf,0.505926
7,91.836735,linear,0.499259
8,255.102041,linear,0.528148
9,448.979592,linear,0.528889


In [58]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

Choosing the best model

In [64]:
model_params = {
    'svm':{
        'model': svm.SVC(gamma='auto'),
        'params': {
            'C': np.linspace(0,500,50),
            'kernel': ['rbf','linear']
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [0,5,10]
        }
    },
    'logistic_regression': {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': np.linspace(0,500,50)
        }
    }
}

In [65]:
scores = []

for model_name, mp in model_params.items():
    clf = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(X, df['type_of_opt'])
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

/Users/saleksegid/miniconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/saleksegid/miniconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/saleksegid/miniconda3/lib/python3.8/site-packages/sklearn/svm/_base.py", line 226, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "/Users/saleksegid/miniconda3/lib/python3.8/site-packages/sklearn/svm/_base.py", line 277, in _dense_fit
    self._probB, self.fit_status_ = libsvm.fit(
  File "sklearn/svm/_libsvm.pyx", line 192, in sklearn.svm._libsvm.fit
ValueError: C <= 0

  warnings.warn("Estimator fit failed. The score on this train-test"
/Users/saleksegid/miniconda3/lib/python3.8/site-packages/

/Users/saleksegid/miniconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.42740741 0.41333333 0.45407407 0.43555556
 0.46222222 0.45481481 0.46888889 0.46296296 0.46962963 0.47481481
 0.47555556 0.47777778 0.47703704 0.48518519 0.48       0.49481481
 0.48518519 0.49925926 0.48962963 0.50666667 0.49111111 0.51407407
 0.49259259 0.52074074 0.49333333 0.53111111 0.49333333 0.52888889
 0.49703704 0.52814815 0.49777778 0.52814815 0.49925926 0.52962963
 0.49703704 0.52888889 0.50074074 0.52666667 0.50222222 0.52888889
 0.50518519 0.52962963 0.50666667 0.52740741 0.50518519 0.52814815
 0.50814815 0.52740741 0.50592593 0.52814815 0.50962963 0.52962963
 0.51185185 0.53185185 0.51555556 0.53111111 0.5162963  0.53111111
 0.5162963  0.53185185 0.51851852 0.53111111 0.52148148 0.52962963
 0.52222222 0.52740741 0.5237037  0.52666667 0.5237037  0.52592593
 0.52296296 0.52518519 0.52296296 0.531

/Users/saleksegid/miniconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan 0.39925926 0.41777778 0.42888889 0.43259259 0.43333333
 0.43925926 0.44148148 0.4437037  0.44666667 0.45111111 0.45185185
 0.45333333 0.45555556 0.45481481 0.4562963  0.4562963  0.45481481
 0.4562963  0.45777778 0.4562963  0.45259259 0.45555556 0.45703704
 0.45777778 0.45777778 0.45777778 0.45703704 0.45777778 0.45703704
 0.45851852 0.45925926 0.46       0.45925926 0.45925926 0.46
 0.46222222 0.46296296 0.4637037  0.4637037  0.4637037  0.4637037
 0.4637037  0.46444444 0.46444444 0.46444444 0.46666667 0.46666667
 0.46592593 0.46518519]
  warnings.warn(


In [66]:
df_2 = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df_2

,model,best_score,best_params
0,svm,0.531852,"{'C': 275.51020408163265, 'kernel': 'linear'}"
1,random_forest,0.519259,{'n_estimators': 10}
2,logistic_regression,0.466667,{'C': 469.3877551020408}


In [67]:
### abovel showes that svm has the higher score, therefore the better model

In [20]:
def kneighbors_graph(self):
    self.X_train = self.X_train.values
    A = neighbors.kneighbors_graph(self.X_train, 9, 'distance')
    plt.spy(A)
    plt.show

In [21]:
X = X
y = y
h = .02

cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#00AAFF'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#00AAFF'])

clf = neighbors.KNeighborsClassifier(K_neighbors, weights='distance')
clf.fit(X, y)

x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

Z = Z.reshape(xx.shape)
plt.figure()
plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold)
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.title("1-Class classification (k = %i)" % (K_neighbors))
plt.show()

TypeError: '(slice(None, None, None), 0)' is an invalid key